# Implementation of Transformers from Scratch

**Reference**
- Paper : [Attention is all you need (Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin, arxiv, 2017)](https://arxiv.org/pdf/1706.03762.pdf)
- Code : [Pytorch Transformers from Scratch (Attention is all you need) by Aladdin Persson](https://youtu.be/U0s0f995w14)

![transformer](https://user-images.githubusercontent.com/59340911/136672443-fa2b2cca-131b-4bda-aa8a-63a252d2a291.png)
The full model architecture of the transformer. (Image source: Fig 1 & 2 in Vaswani, et al., 2017.)

In [1]:
import torch
import torch.nn as nn

# 1. Self-Attention
![self_attention](https://user-images.githubusercontent.com/59340911/136672447-5dc8abfe-3208-4061-a0a7-730e4e9781d6.png)

In [2]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        """
        Split embeddings into different parts (= # of heads)
        ex. embed_size = 256, # of heads = 8 => 8 * 32(= head_dim)
        """
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads
        
        assert (self.head_dim * heads == embed_size), "Embed size needs to be div by heads"
        
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads*self.head_dim, embed_size) # heads*head_dim == embed_size
        
    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        
        # Split embeddings into self.heads pieces
        # (N, L, E) -> (N, L, heads, head_dim)
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)
        
        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)
        
        """
        Matrix Multiplication
        queries : (N, query_len, heads, head_dim)
        keys : (N, key_len, heads, head_dim)
        => energy : (N, heads, query_len, key_len)
        """
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        
        if mask is not None:
            # if the element of the mask equals zero, we make it have no impact on others
            # by replacing it with a very very small value
            energy = energy.masked_fill(mask == 0, float("-1e20"))
        
        attention = torch.softmax(energy / (self.embed_size ** (1/2)), 
                                  dim=3) # dim=3 : normalize across the key length
        
        """
        attention : (N, heads, query_len, key_len)
        values : (N, value_len, heads, head_dim)
        => out : (N, query_len, heads, head_dim) = (N, query_len, embed_size) 
        """
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape( # key_len == value_len
            N, query_len, self.heads*self.head_dim
        )
        out = self.fc_out(out)
        
        return out

# 2. Transformer Block
![transformer_block](https://user-images.githubusercontent.com/59340911/136672456-c81f4de1-5818-42c9-9297-b4633d810335.png)

![ffn](https://user-images.githubusercontent.com/59340911/136672462-a6026cff-7456-485e-a273-f4f84d3ef93c.png)

In [3]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        
        # don't change anything, just more computation and map it back
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion*embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion*embed_size, embed_size)
        )
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

# 3. Encoder
![encoder](https://user-images.githubusercontent.com/59340911/136672466-a73840b7-bd34-41e5-bf70-23a292e45262.png)

In [4]:
class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length # related to positional embedding
    ):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)
        
        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion
                )
                for _ in range(num_layers)
            ]
        )
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        N, seq_length = x.shape # (N, L)
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))
        
        for layer in self.layers:
            out = layer(out, out, out, mask) # q == k == v
        
        return out

![positions](https://user-images.githubusercontent.com/59340911/136672473-3111c32b-39f3-4c0c-8e20-945b390d8857.png)

In [5]:
# for better understanding
positions = torch.arange(0, 10).expand(5, 10) # (1, 10) -> (5, 10)
print(torch.arange(0, 10).shape) 
positions

torch.Size([10])


tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

# 4. Decoder Block
![decoder_block](https://user-images.githubusercontent.com/59340911/136672478-4d51d95a-9cf5-4d5b-a413-7f0ba6429a90.png)

In [6]:
class DecoderBlock(nn.Module):
    def __init__(
        self,
        embed_size,
        heads,
        forward_expansion,
        dropout,
        device
    ):
        super(DecoderBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        self.transformer_block = TransformerBlock(
            embed_size,
            heads,
            dropout,
            forward_expansion
        )
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, value, key, src_mask, trg_mask):
        """
        src_mask : Optional
        To make all examples have equal length, we add a [PAD] token.
        If we use src_mask, we don't do unnecessary computations for tokens that are padded.
        """
        attention = self.attention(x, x, x, trg_mask) # masked mha
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out

# 5. Decoder
![decoder](https://user-images.githubusercontent.com/59340911/136672489-0b92cf05-9244-4037-82d4-e379122ef684.png)

In [7]:
class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)
        
        self.layers = nn.ModuleList(
            [
                DecoderBlock(
                    embed_size,
                    heads,
                    forward_expansion,
                    dropout,
                    device
                )
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
       
    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout(self.word_embedding(x) + self.position_embedding(positions))
        
        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)
            
        out = self.fc_out(x)
        return out

# 6. Transformer
![transformer](https://user-images.githubusercontent.com/59340911/136672443-fa2b2cca-131b-4bda-aa8a-63a252d2a291.png)

In [8]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=256,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="cuda",
        max_length=100
    ):
        super(Transformer, self).__init__()
        
        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length
        )
        
        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length
        )
        
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2) # (N, src_len) -> (N, 1, 1, src_len)
        return src_mask.to(self.device)
    
    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand( # (trg_len, trg_len) -> (N, 1, trg_len, trg_len)
            N, 1, trg_len, trg_len
        )
        return trg_mask.to(self.device)
    
    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

- Why "(N, src_len) -> (N, 1, 1, src_len)" & "(trg_len, trg_len) -> (N, 1, trg_len, trg_len)"?
    - Because we use masks for the vector which dimension is (N, heads, query_len, key_len).
    - Image Source : https://github.com/paul-hyun/transformer-evolution

![encoder](https://user-images.githubusercontent.com/59340911/136672425-d96b9967-7408-4cf0-9908-f0d47a6237ef.png)

![decoder](https://user-images.githubusercontent.com/59340911/136672426-cb91b8cf-0042-4d3d-a081-a6c48585c89f.png)

# 7. How It Works

In [9]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(device)
    trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

    src_pad_idx = 0
    trg_pad_idx = 0
    src_vocab_size = 10
    trg_vocab_size = 10
    model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(
        device
    )
    out = model(x, trg[:, :-1])
    print(out)
    print(out.shape)

cuda
tensor([[[-0.2093,  0.0763, -0.6496,  0.6595,  0.3369,  0.0987,  0.5505,
          -0.8110, -0.5675, -0.5306],
         [ 0.1050,  0.2950, -0.3066,  0.0709, -0.1792,  0.2724,  0.7345,
          -0.3551, -0.4767,  0.1399],
         [-1.5266, -0.1127,  0.2552,  0.0400,  0.4366, -0.0451,  0.2982,
          -0.7908, -1.2410, -0.8053],
         [-0.2560,  0.2025, -0.8274, -0.9275,  0.3588, -0.2666,  0.6953,
          -0.4531, -1.9198,  1.0471],
         [-0.1886,  0.1243, -1.2540, -0.2852,  0.1705,  0.1605, -0.4975,
          -0.4545, -1.0439, -0.6507],
         [ 0.2705,  0.0466, -0.9329,  0.0520, -0.0092, -0.1894,  1.0094,
          -1.0697, -1.8518,  0.5283],
         [ 0.2934,  0.0611, -1.3078, -0.9980,  0.0581,  0.2332,  0.0848,
          -1.1099, -0.4155,  0.9614]],

        [[-0.1702,  0.0468, -0.7531,  0.6788,  0.1154,  0.0813,  0.4585,
          -1.0972, -0.4126, -0.6720],
         [-0.4773,  0.7665, -0.8518,  0.2010,  0.2601,  0.5256,  0.0100,
          -0.0092,  0.0574, -0.2